In [ ]:
import cv2, os, joblib
import numpy as np
from pathlib import Path
from keras.models import load_model

In [ ]:
# Load HAAR face classifier
face_classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

# Load functions
def face_extractor(img):
    # Function detects faces and returns the cropped face
    # If no face detected, it returns the input image
    
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray, 1.3, 5)
    
    if faces == ():
        return None
    
    # Crop all faces found
    for (x,y,w,h) in faces:
        cropped_face = img[y:y+h, x:x+w]

    return cropped_face



In [ ]:
class Kev:
    def file(self, person_name):
        train_dir = Path('C://Users//kevin//Documents//Project//Task_4//Train//')
        self.A = train_dir / person_name
        os.mkdir(self.A)
        test_dir = Path('C://Users//kevin//Documents//Project//Task_4//Validation//')
        self.B = test_dir / person_name
        os.mkdir(self.B)

    def capture(self):
        # Initialize Webcam
        cap = cv2.VideoCapture(0)
        count = 0


        # Collect 100 samples of your face from webcam input
        while True:

            ret, frame = cap.read()
            if face_extractor(frame) is not None:
                count += 1
                face = cv2.resize(face_extractor(frame), (200, 200))
                face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)

                # Save file in specified directory with unique name for train and test Each

                file_name_train_path = str(self.A)+ '/' + str(count) + '.jpg'
                cv2.imwrite(file_name_train_path, face)
                file_name_test_path = str(self.B) + '/' + str(count) + '.jpg'
                cv2.imwrite(file_name_test_path, face)

                # Put count on images and display live count
                cv2.putText(face, str(count), (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
                cv2.imshow('Face Cropper', face)

            else:
                print("Face not found")
                pass

            if cv2.waitKey(1) == 13 or count == 100: #13 is the Enter Key
                break

        cap.release()
        cv2.destroyAllWindows()      
        print("Collecting Samples Complete")


In [ ]:
def Learning():
    import cv2
    from pathlib import Path
    from keras.models import Model, Sequential
    from keras.preprocessing import image
    from keras.applications import Xception
    from keras.preprocessing.image import ImageDataGenerator
    from keras.layers import Dense, Conv2D, GlobalMaxPooling2D, Dropout, Activation, Flatten
    from keras.optimizers import RMSprop


    img_rows, img_cols = 224, 224 
    model = Xception(weights = 'imagenet', 
                     include_top = False, 
                     input_shape = (img_rows, img_cols, 3))


    # Here we freeze the last 4 layers 
    # Layers are set to trainable as True by default
    for layer in model.layers:
        layer.trainable = False
    model.summary()


    def add(BottomModel, Class):
        top_model = BottomModel.output
        top_model = GlobalMaxPooling2D()(top_model)
        top_model = Dense(1024,activation='relu')(top_model)
        top_model = Dense(1024,activation='relu')(top_model)
        top_model = Dense(512,activation='relu')(top_model)
        top_model = Dense(Class,activation='softmax')(top_model)
        return top_model


    Class = 2
    top = add(model, Class)
    model = Model(inputs = model.input, outputs = top)


    model.summary()


    #Image Data Generator
    train_datagen = ImageDataGenerator(
            rescale=1./255,
            shear_range=0.2,
            zoom_range=0.2,
            horizontal_flip=True)
    validation_datagen = ImageDataGenerator(rescale=1./255)
    test_datagen = ImageDataGenerator(rescale=1./255)
    train_generator = train_datagen.flow_from_directory(
            'C://Users//kevin//Documents//Project//Task_4//Train//',
            target_size=(img_rows, img_cols),
            batch_size=64,
            class_mode='categorical')
    validation_generator = validation_datagen.flow_from_directory(
            'C://Users//kevin//Documents//Project//Task_4//Validation//',
            target_size=(img_rows, img_cols),
            batch_size=64,
            class_mode='categorical')


    model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
    #validation_steps < validation_dataset_size / batch_size
    model.fit(
            train_generator,
            epochs=5,
            validation_data=validation_generator)


    model.save('Face_Detector.h5')

In [ ]:
k = Kev()
ans=True
while ans:
    print("""
    1. Add Face Image Data
    2. Train Model
    3. Test Model
    4.Exit/Quit
    """)
    ans=input("What would you like to do? ")
    if ans=="1":
        k.file(person_name = input("Name?"))
        k.capture()
        print("\nFace Data Added")
    elif ans=="2":
        Learning()
        print("\nModel Trained")
    elif ans=="3":
        model = load_model('Face_Detector.h5')
        print("\n Student Record Found")
    elif ans=="4":
      print("\n Goodbye") 
      ans = None
    else:
       print("\n Not Valid Choice Try again")

In [ ]:
print('Yay, the file exists!') if k.A.exists() and k.B.exists() else print("Oops, file doesn't exist!")

In [ ]:
joblib.load('Face Detector.h5')